first lets upload the dataset

In [1]:
from google.colab import files
uploaded = files.upload()

Saving iris.csv to iris.csv


Load and read the data set

In [4]:
import pandas as pd
df = pd.read_csv('iris.csv')
print(df.head())

   SepalLength  SepalWidth  PetalLength  PetalWidth     Species 
0          5.1         3.5          1.4         0.2  Iris-setosa
1          4.9         3.0          1.4         0.2  Iris-setosa
2          4.7         3.2          1.3         0.2  Iris-setosa
3          4.6         3.1          1.5         0.2  Iris-setosa
4          5.0         3.6          1.4         0.2  Iris-setosa


Checking for the missing value

In [5]:
df.isnull().sum()

,0
SepalLength,0
SepalWidth,0
PetalLength,0
PetalWidth,0
Species,0


No missing value. Displaying input and output features of the dataset.

In [8]:
print("\nInput features:")
print(df.columns[:-1].tolist())

print("\nOutput feature:")
print(df.columns[-1])


Input features:
['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

Output feature:
Species 


Data sample

In [9]:
df.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Encoding output attribute using one hot encoder

In [21]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)

y = ohe.fit_transform(df[['Species ']])

print(y[:5])

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


Shuffling the Dataset and counting number of tuples in each class

In [23]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

df['Species '].value_counts()

,count
Species,
Iris-virginica,50
Iris-versicolor,50
Iris-setosa,50


Normalizing input attributes using Standard Scaler

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('Species ', axis=1))

Splitting dataset into training/validation/test sets in 70:15:15 ratio.

In [29]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42)

Contructing an MLP with configuration 4x32x16x8x3. Using Adam optimizer, training the model.

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential([
    Input(shape=(4,)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.3495 - loss: 1.1144 - val_accuracy: 0.3478 - val_loss: 1.1137
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3954 - loss: 1.1034 - val_accuracy: 0.3478 - val_loss: 1.1078
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3744 - loss: 1.1054 - val_accuracy: 0.2609 - val_loss: 1.1039
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3882 - loss: 1.0906 - val_accuracy: 0.2174 - val_loss: 1.1013
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3451 - loss: 1.0979 - val_accuracy: 0.2609 - val_loss: 1.0996
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3526 - loss: 1.0986 - val_accuracy: 0.3043 - val_loss: 1.0996
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4098 - loss: 1.0936 - val_accuracy: 0.3478 - val_loss: 1.0988
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4610 - loss: 1.0970 - val_accuracy: 0.4348 - val_loss: 1.0961


Predict species of Iris flower for test data and display confusion matrix,
weighted avg. accuracy, macro & micro recall, macro & micro precision and
macro and micro F1-score.


In [35]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)


cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

cr = classification_report(y_true, y_pred_classes)
print(cr)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
[[1 1 6]
 [1 3 5]
 [2 0 4]]
              precision    recall  f1-score   support

           0       0.25      0.12      0.17         8
           1       0.75      0.33      0.46         9
           2       0.27      0.67      0.38         6

    accuracy                           0.35        23
   macro avg       0.42      0.38      0.34        23
weighted avg       0.45      0.35      0.34        23

